In [17]:
import numpy as np
import seaborn as sns
from time import time
import gc

In [18]:
init_state = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
])

In [60]:
class Game:
    def __init__(self, state, FIRST=1):
        self.state = state
        self.empty = self.make_empty(state)
        self.first_player = FIRST
        
    def make_empty(self, state):
        emp = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    emp.append(3*i + j)
        
        return emp
    
    def is_lose(self):
        a = self.next_opp()
        
        for i in range(3):
            if self.state[i][0] == self.state[i][1] == self.state[i][2] == a:
                return True
            elif self.state[0][i] == self.state[1][i] == self.state[2][i] == a:
                return True
        if self.state[0][0] == self.state[1][1] == self.state[2][2] == a:
            return True
        if self.state[0][2] == self.state[1][1] == self.state[2][0] == a:
            return True
        return 0
    
#     def is_win(self):       
#         a = self.next_opp()
#         for i in range(3):
#             if self.state[i][0] == self.state[i][1] == self.state[i][2] == a:
#                 return True
#             elif self.state[0][i] == self.state[1][i] == self.state[2][i] == a:
#                 return True
#         if self.state[0][0] == self.state[1][1] == self.state[2][2] == a:
#             return True
#         if self.state[0][2] == self.state[1][1] == self.state[2][0] == a:
#             return True
        
#         return False
    
    def is_draw(self):
        a = self.next_opp()
        if self.is_lose():
            return 0
        if np.all(self.state):
            return 1
        else:
            return 0
        
    def is_done(self):
        if self.is_lose() or self.is_draw():
            return 1
        else:
            return 0
        
        
    def update(self, target):
        state = self.state.copy()
        x, y = target//3, target%3
        a = self.next_opp()
        state[x][y] = a
        return Game(state)
    
    
    def next_opp(self):
        a = b = 0
        for i in range(len(self.state)):
            for j in range(len(self.state)):
                if self.state[i][j] == self.first_player:
                    a += 1
                elif self.state[i][j] != 0:
                    b += 1
                    
        if a == b:
            return self.first_player
        else:
            return 2 + min(0, 1-self.first_player)

In [61]:
class Random:
    def action(self, game):
        return np.random.choice(game.empty)

In [73]:
class Alpha:
    def value(self, game, alpha, beta):
#         print(game.state)
        if game.is_lose():
            return -1
        
        if game.is_draw():
            return 0
        
#         if game.is_win():
# #             print(game.state)
#             return 1
        
        
        best_score = -float('inf')
        score = 0
        for a in game.empty:
            score -= self.value(game.update(a), beta, alpha)
            
            if score > alpha:
                alpha = score
#                 print(score)

            if alpha >= beta:
                return alpha
                
        return alpha
        
    def action(self, game):
#         if FIRST == self.status and len(game.empty) == 9:
#             return 4
        
        alpha = -float('inf')
        
        for a in game.empty:
            score = self.value(game.update(a), -float('inf'), alpha)
            if score > alpha:
                best_action = a
                alpha = score
#                 print(best_action)
        return best_action

    

In [93]:
def play(game, m1, m2):
    global score
    while 1:
        a1 = m1.action(game)
        game = game.update(a1)
#         print(game.state)
        if game.is_lose():
            score[0] += 1
#             print(game.state)
            return 
        elif game.is_draw():
            score[2] += 1
#             print(game.state)
            return 

        a2 = m2.action(game)
        game = game.update(a2)
#         print(game.state)
        if game.is_lose():
            score[1] += 1
#             print(game.state)
            return 
        elif game.is_draw():
            score[2] += 1
#             print(game.state)
            return 
        

In [94]:
# FIRST = 1
game = Game(init_state)
m1 = Random()
m2 = Alpha()

In [96]:
%%time
score = [0, 0, 0]
for _ in range(1000):
#     print(_)
    play(game, m1, m2)
    
print(score)
gc.collect()

score = [0, 0, 0]
for _ in range(1000):
#     print(_)
    play(game, m2, m1)
print(score)

[651, 176, 173]
[59, 910, 31]
Wall time: 7min 14s


In [ ]:
# sns.barplot(x = [1, 2], y = score[:2])

In [ ]:
# v1: 75vs120
# v2: 57vs134
# v3: 61vs128
# v4: 52vs131 // 33vs56
# v5: 47vs144 // 29vs65
# v6: 45vs150 // 32vs63
# v7 221vs1183 // 113vs402
# v8 310vs1210 // 78vs492
# v9 331vs1444 // 194vs645
# v10 235vs1561 // complete